In [1]:
!pip install pandas xlsxwriter

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()


25/03/04 06:04:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark.sql("DROP TABLE IF EXISTS demo.nyc.taxis_10M_50COLUMNS_batch")


DataFrame[]

In [4]:
import os
iceberg_table_dir = "../warehouse/nyc/taxis_10M_50COLUMNS_batch"
metadata_dir = f"{iceberg_table_dir}/metadata"
data_dir = f"{iceberg_table_dir}/data"
input_data_dir = f"../input_data"
analysis_info = []
records_before_op = 0

def append_to_file(file_path, msg):
    open_mode = "a"
    if not os.path.exists(file_path):
        open_mode = "w"

    # Open the CSV file in write mode
    with open(file_path, open_mode) as file:
        writer = csv.writer(file)
        
        if open_mode=="w":
            #writing header of the columns
            writer.writerows([list(msg.keys())])    

        row_values = [list(msg.values())]
        # Write the data to the CSV file
        writer.writerows(row_values)

def get_size():
    # List the metadata files
    manifest_pattern = re.compile(r".*-m\d+\.avro$")
    metadata_files = os.listdir(metadata_dir)
    
    # Initialize variables to store the sizes of different types of metadata files
    snap_avro_size = 0
    metadata_json_size = 0
    m_avro_size = 0

    data_dir_size = 0
    # get data dir size
    data_dir_files = os.listdir(data_dir)
    # print(data_dir_files)
    for filename in data_dir_files:
        file_path = os.path.join(data_dir, filename)
        data_dir_size += os.path.getsize(file_path) / 1024  # Convert size to KB
    
    # Iterate through the metadata files and calculate their sizes
    for file in metadata_files:
        file_path = os.path.join(metadata_dir, file)
        file_size_kb = os.path.getsize(file_path) / 1024  # Convert size to KB
        
        if file.startswith("snap-") and file.endswith(".avro"):
            snap_avro_size += file_size_kb
        elif file.endswith(".metadata.json"):
            metadata_json_size += file_size_kb
        elif manifest_pattern.match(file):
            m_avro_size += file_size_kb
    
    # Print the time taken and the sizes of the metadata files
    # print(f"Time taken to read Parquet files: {time_taken:.2f} seconds")
    # print(f"Size of snap-*.avro files: {snap_avro_size:.2f} KB")
    # print(f"Size of *.metadata.json files: {metadata_json_size:.2f} KB")
    # print(f"Size of *m{0-9}{1,}.avro files: {m_avro_size:.2f} KB")

    return {"data_dir_size": data_dir_size,"metadata_size": metadata_json_size,"snapshot_size": snap_avro_size,"manifest_size": m_avro_size}


In [5]:
from pyspark.sql.types import (
    DoubleType, FloatType, LongType, StructType, StructField, 
    StringType, IntegerType, DateType
)

# Define the schema with 50 columns based on the required data types
schema = StructType([
    # StructField("vendor_id", LongType(), True),  # INT
    # StructField("trip_id", LongType(), True),  # INT
    # StructField("trip_distance", FloatType(), True),  # FLOAT
    # StructField("fare_amount", DoubleType(), True),  # DOUBLE
    # StructField("store_and_fwd_flag", StringType(), True)  # STRING
# ] + [
    # Assigning VARCHAR, INT, STRING, and DATE data types in a cyclic pattern
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 0 else  # VARCHAR
    StructField(f"extra_col_{i}", IntegerType(), True) if i % 4 == 1 else  # INT
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 2 else  # STRING
    StructField(f"extra_col_{i}", DateType(), True)  # DATE
    for i in range(50)
])

# Create an empty DataFrame with the schema
df = spark.createDataFrame([], schema)

# Create the Iceberg table
df.writeTo("demo.nyc.taxis_10M_50COLUMNS_batch").create()


25/03/04 06:04:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
df = spark.table("demo.nyc.taxis_10M_50COLUMNS_batch")
df.show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|extra_col_0|extra_col_1|extra_col_2|extra_col_3|extra_col_4|extra_col_5|extra_col_6|extra_col_7|extra_col_8|extra_col_9|extra_col_10|extra_col_11|extra_col_12|extra_col_13|extra_col_14|extra_col_15|extra_col_16|extra_col_17|extra_col_18|extra_col_19|extra_col_20|extra_col_21|extra_col_22|extra_col_23|extra_col_24|extra_col_25|extra_col_26|extra_col_27|ext

In [7]:
import time, csv
from pyspark.sql import functions as F
import os

input_data_dir = "../input_data"
output_dir = "../output"
analysis_info = []
records_before_op = 0
total_insertion_time = 0

file_type = input("Enter input file type csv or parquet? : ").lower().strip()
input_data_dir = os.path.join(input_data_dir, file_type)
input_files = os.listdir(input_data_dir)

analysis_file = os.path.join(output_dir, f"analysis_info_{file_type}.csv")
if os.path.exists(analysis_file):
    os.remove(analysis_file)

df = spark.table("demo.nyc.taxis_10M_50COLUMNS_batch")
records_before_op = df.count()

# Define batch size (e.g., 100,000 records per batch)
batch_size = 100000

for file in input_files:
    print(f"Started with file={file}")
    file_path = os.path.join(input_data_dir, file)

    st = time.time()
    if file_type == "parquet":
        # Read the parquet file in batches
        input_df = spark.read.parquet(file_path)
    else:
        input_df = spark.read.csv(file_path, header=True)
        input_df = input_df.select(
            F.col("extra_col_0").cast("long"),
            F.col("extra_col_1").cast("int"),
            F.col("extra_col_2").cast("string"),
            F.col("extra_col_3").cast("date"),
            *[F.col(f"extra_col_{i}").cast("string" if i % 4 == 0 or i % 4 == 2 else "int" if i % 4 == 1 else "date") for i in range(4, 45)]
        )
    
    rows = input_df.count()
    
    # Processing in batches
    num_batches = (rows // batch_size) + 1
    for batch_num in range(num_batches):
        batch_start = batch_num * batch_size
        batch_end = (batch_num + 1) * batch_size
        
        # Get the batch of records (limit and offset logic for batch processing)
        batch_df = input_df.limit(batch_end).subtract(input_df.limit(batch_start))
        
        # Perform operations on this batch (e.g., append to the table)
        batch_df.writeTo("demo.nyc.taxis_10M_50COLUMNS_batch").append()

        # Track insertion time for this batch
        end = time.time() - st
        total_insertion_time += end
        records_before_op += batch_df.count()

        print(f"Inserted {batch_df.count()} records from batch {batch_num+1}/{num_batches} in {end:.2f} sec.")
    
# **PRINT INSERTION TIME BEFORE UPDATE**
print(f"\nTotal insertion time: {total_insertion_time:.2f} sec\n")

# === Get Update Percentage from User ===
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100

df = spark.table("demo.nyc.taxis_10M_50COLUMNS_batch")
total_rows = df.count()

num_rows = int(total_rows * update_percentage)
print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# Sample random rows ensuring unique IDs
sampled_df = df.select("extra_col_0").distinct().orderBy(F.rand()).limit(num_rows)
sampled_ids = [row["extra_col_0"] for row in sampled_df.collect()]

st = time.time()

# Updating `extra_col_1` (integer)
updated_df = df.withColumn(
    "extra_col_1",
    F.when(F.col("extra_col_0").isin(sampled_ids), F.col("extra_col_1") + 10)
    .otherwise(F.col("extra_col_1"))
)

# Overwrite updated data in batches
updated_df.writeTo("demo.nyc.taxis_10M_50COLUMNS_batch").overwritePartitions()

end = time.time() - st

print(f"Updated {num_rows} rows in {end:.2f} sec")


Enter input file type csv or parquet? :  parquet


Started with file=records_1000000_part_10_1740401457.66906.parquet


Inserted 100000 records from batch 1/11 in 14.89 sec.


25/03/04 06:05:16 ERROR Executor: Exception in task 1.0 in stage 17.0 (TID 28)4]
java.lang.OutOfMemoryError: Java heap space
	at org.apache.parquet.bytes.BytesInput$StreamBytesInput.toByteArray(BytesInput.java:285)
	at org.apache.parquet.bytes.BytesInput.toByteBuffer(BytesInput.java:237)
	at org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainBinaryDictionary.<init>(PlainValuesDictionary.java:91)
	at org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainBinaryDictionary.<init>(PlainValuesDictionary.java:74)
	at org.apache.parquet.column.Encoding$1.initDictionary(Encoding.java:90)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader.<init>(VectorizedColumnReader.java:123)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.initColumnReader(VectorizedParquetRecordReader.java:437)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.checkEndOfRowGroup(VectorizedPar

ConnectionRefusedError: [Errno 111] Connection refused

In [77]:
spark.sql("SELECT * FROM demo.nyc.taxis_1M_50COLUMNS.history")


DataFrame[made_current_at: timestamp, snapshot_id: bigint, parent_id: bigint, is_current_ancestor: boolean]

In [84]:
df = spark.sql("SELECT snapshot_id, made_current_at FROM demo.nyc.taxis_1M_50COLUMNS.history ORDER BY made_current_at DESC")

df.show(

+-------------------+--------------------+
|        snapshot_id|     made_current_at|
+-------------------+--------------------+
|2861666654166238235|2025-02-25 13:10:...|
|7867670594422745254|2025-02-25 13:09:...|
+-------------------+--------------------+



In [ ]:
WITH current_snapshot AS (
    SELECT * FROM demo.nyc.taxis_1M_50COLUMNS.snapshot(2861666654166238235)
),
previous_snapshot AS (
    SELECT * FROM demo.nyc.taxis_1M_50COLUMNS.snapshot(7867670594422745254)
)
SELECT p.*, c.*
FROM previous_snapshot p
FULL OUTER JOIN current_snapshot c
ON p.primary_key_column = c.primary_key_column
WHERE p.primary_key_column IS NULL OR c.primary_key_column IS NULL;


In [85]:
spark.sql("DESCRIBE demo.nyc.taxis_1M_50COLUMNS").show()


+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
| extra_col_0|   string|   NULL|
| extra_col_1|      int|   NULL|
| extra_col_2|   string|   NULL|
| extra_col_3|     date|   NULL|
| extra_col_4|   string|   NULL|
| extra_col_5|      int|   NULL|
| extra_col_6|   string|   NULL|
| extra_col_7|     date|   NULL|
| extra_col_8|   string|   NULL|
| extra_col_9|      int|   NULL|
|extra_col_10|   string|   NULL|
|extra_col_11|     date|   NULL|
|extra_col_12|   string|   NULL|
|extra_col_13|      int|   NULL|
|extra_col_14|   string|   NULL|
|extra_col_15|     date|   NULL|
|extra_col_16|   string|   NULL|
|extra_col_17|      int|   NULL|
|extra_col_18|   string|   NULL|
|extra_col_19|     date|   NULL|
+------------+---------+-------+
only showing top 20 rows

